In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

%config InlineBackend.figure_format = 'svg'
%matplotlib inline 

# Download and load MTA turnstile data

In [2]:
def all_saturdays(start_date, end_date):
    modified_end_date = datetime.datetime.strptime(end_date,'%m/%d/%Y')+ datetime.timedelta(weeks = 1)
    saturday_list = pd.date_range(start =start_date, end =modified_end_date,
                              freq='W-SAT').strftime('%m/%d/%Y').tolist()
    return saturday_list

In [3]:
def import_mta(date):
    formatted_date = date[-2:]+date[:2]+date[3:5]
    base_url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    date_url = base_url.format(formatted_date)
    date_data = pd.read_csv(date_url)
    return date_data

In [4]:
def mta_to_df(saturday_list):
    dict_of_dfs = {}
    for saturday in saturday_list:
        dict_of_dfs[saturday] = pd.DataFrame(import_mta(saturday))
    mta =pd.DataFrame()
    for val in dict_of_dfs.values():
        mta = pd.concat([mta, val])
    return mta

In [5]:
start_date = input("Enter start date (X/X/XXXX): ")
end_date = input("Enter end date (X/X/XXXX): ")

Enter start date (X/X/XXXX): 3/1/2019
Enter end date (X/X/XXXX): 6/1/2019


In [6]:
saturday_list = all_saturdays(start_date,end_date)

In [ ]:
# print(saturday_list)

In [8]:
mta = mta_to_df(saturday_list)

In [9]:
mta.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
205006,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/05/2019,17:00:00,RECOVR AUD,5554,379
205007,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/05/2019,21:00:00,REGULAR,5554,379
205008,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/06/2019,01:00:00,REGULAR,5554,379
205009,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/06/2019,05:00:00,REGULAR,5554,379
205010,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/06/2019,09:00:00,REGULAR,5554,379


# In case of later breakage

In [10]:
mta.to_csv(r'mta.csv')

In [32]:
# mta = pd.read_csv('mta.csv')

# Cleaning

## column names

In [33]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3062159 entries, 0 to 3062158
Data columns (total 12 columns):
Unnamed: 0                                                              int64
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                 int64
EXITS               

In [34]:
mta.columns

Index(['Unnamed: 0', 'C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION',
       'DATE', 'TIME', 'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [35]:
mta.columns = mta.columns.str.strip().str.lower().str.replace('/',"_")

In [36]:
mta.columns

Index(['unnamed: 0', 'c_a', 'unit', 'scp', 'station', 'linename', 'division',
       'date', 'time', 'desc', 'entries', 'exits'],
      dtype='object')

## create datetime column from date and time

In [37]:
mta["datetime"] = pd.to_datetime(mta.date + " " + mta.time, format="%m/%d/%Y %H:%M:%S")

In [38]:
mta.head()

,unnamed: 0,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,datetime
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/23/2019,03:00:00,REGULAR,6955483,2359112,2019-02-23 03:00:00
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/23/2019,07:00:00,REGULAR,6955494,2359125,2019-02-23 07:00:00
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/23/2019,11:00:00,REGULAR,6955554,2359199,2019-02-23 11:00:00
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/23/2019,15:00:00,REGULAR,6955714,2359248,2019-02-23 15:00:00
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/23/2019,19:00:00,REGULAR,6956004,2359292,2019-02-23 19:00:00


## filter to previously specified dates

In [39]:
mta.date.value_counts().sort_index()

02/23/2019    28827
02/24/2019    29109
02/25/2019    28842
02/26/2019    29777
02/27/2019    28903
              ...  
06/03/2019    30022
06/04/2019    29513
06/05/2019    29484
06/06/2019    29213
06/07/2019    28745
Name: date, Length: 105, dtype: int64

In [40]:
mta.date = pd.to_datetime(mta.date,format="%m/%d/%Y")

In [41]:
mta = mta[(mta.date >= start_date)]

In [42]:
mta = mta[(mta.date <= end_date)]

In [43]:
mta.date.value_counts().sort_index()

2019-03-01    28691
2019-03-02    29234
2019-03-03    28836
2019-03-04    29067
2019-03-05    29234
              ...  
2019-05-28    28873
2019-05-29    29095
2019-05-30    29190
2019-05-31    29651
2019-06-01    29066
Name: date, Length: 93, dtype: int64

In [44]:
mta.drop(columns = ['time'], inplace = True)

In [45]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2711716 entries, 36 to 3062131
Data columns (total 12 columns):
unnamed: 0    int64
c_a           object
unit          object
scp           object
station       object
linename      object
division      object
date          datetime64[ns]
desc          object
entries       int64
exits         int64
datetime      datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(7)
memory usage: 269.0+ MB


In [46]:
mta.head(10)

,unnamed: 0,c_a,unit,scp,station,linename,division,date,desc,entries,exits,datetime
36,36,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962094,2361672,2019-03-01 03:00:00
37,37,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962111,2361714,2019-03-01 07:00:00
38,38,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962243,2362017,2019-03-01 11:00:00
39,39,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962502,2362082,2019-03-01 15:00:00
40,40,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6963303,2362149,2019-03-01 19:00:00
41,41,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6963578,2362196,2019-03-01 23:00:00
78,78,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209663,1392692,2019-03-01 03:00:00
79,79,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209673,1392711,2019-03-01 07:00:00
80,80,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209797,1392838,2019-03-01 11:00:00
81,81,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209986,1392887,2019-03-01 15:00:00


## duplicates

In [48]:
(mta
 .groupby(["c_a", "unit", "scp", "station", "datetime"])
 .entries.count()
 .reset_index()
 .sort_values("entries", ascending=False)).head(50)

,c_a,unit,scp,station,datetime,entries
716178,N071,R013,00-06-00,34 ST-PENN STA,2019-04-20 08:00:00,2
2215449,R283,R221,00-00-01,167 ST,2019-04-16 04:00:00,2
1325112,N519,R461,00-03-01,B'WAY-LAFAYETTE,2019-03-26 13:00:00,2
1325111,N519,R461,00-03-01,B'WAY-LAFAYETTE,2019-03-26 09:00:00,2
535277,JFK03,R536,00-00-02,JFK JAMAICA CT1,2019-04-07 01:00:00,2
637616,N045,R187,01-00-00,81 ST-MUSEUM,2019-06-01 17:00:00,2
1325110,N519,R461,00-03-01,B'WAY-LAFAYETTE,2019-03-26 05:00:00,2
1325108,N519,R461,00-03-01,B'WAY-LAFAYETTE,2019-03-25 21:00:00,2
1325107,N519,R461,00-03-01,B'WAY-LAFAYETTE,2019-03-25 17:00:00,2
895018,N135,R385,01-03-01,ROCKAWAY BLVD,2019-03-31 13:00:00,2


In [ ]:
# mask = ((mta["c_a"] == "G009") & 
#     (mta["unit"] == "R151") & 
#     (mta["scp"] == "02-00-04") & 
#     (mta["station"] == "CONEY IS-STILLW") &
#     (mta["datetime"] == "2019-05-16 17:00:00"))
# mta[mask].head(10)

In [26]:
mta.sort_values(["c_a", "unit", "scp", "station", "datetime"], inplace=True, ascending=False)
mta.drop_duplicates(subset=["c_a", "unit", "scp", "station", "datetime"], inplace=True)

In [27]:
(mta
 .groupby(["c_a", "unit", "scp", "station", "datetime"])
 .entries.count()
 .reset_index()
 .sort_values("entries", ascending=False)).head(10)

,c_a,unit,scp,station,datetime,entries
0,A002,R051,02-00-00,59 ST,2019-03-01 03:00:00,1
1807759,R141,R031,00-06-00,34 ST-PENN STA,2019-04-26 08:00:00,1
1807761,R141,R031,00-06-00,34 ST-PENN STA,2019-04-26 16:00:00,1
1807762,R141,R031,00-06-00,34 ST-PENN STA,2019-04-26 20:00:00,1
1807763,R141,R031,00-06-00,34 ST-PENN STA,2019-04-27 00:00:00,1
1807764,R141,R031,00-06-00,34 ST-PENN STA,2019-04-27 04:00:00,1
1807765,R141,R031,00-06-00,34 ST-PENN STA,2019-04-27 08:00:00,1
1807766,R141,R031,00-06-00,34 ST-PENN STA,2019-04-27 12:00:00,1
1807767,R141,R031,00-06-00,34 ST-PENN STA,2019-04-27 16:00:00,1
1807768,R141,R031,00-06-00,34 ST-PENN STA,2019-04-28 00:00:00,1


In [49]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2711716 entries, 36 to 3062131
Data columns (total 12 columns):
unnamed: 0    int64
c_a           object
unit          object
scp           object
station       object
linename      object
division      object
date          datetime64[ns]
desc          object
entries       int64
exits         int64
datetime      datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(7)
memory usage: 269.0+ MB


## daily entries by turnstile

In [50]:
mta.head(10)

,unnamed: 0,c_a,unit,scp,station,linename,division,date,desc,entries,exits,datetime
36,36,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962094,2361672,2019-03-01 03:00:00
37,37,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962111,2361714,2019-03-01 07:00:00
38,38,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962243,2362017,2019-03-01 11:00:00
39,39,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6962502,2362082,2019-03-01 15:00:00
40,40,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6963303,2362149,2019-03-01 19:00:00
41,41,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6963578,2362196,2019-03-01 23:00:00
78,78,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209663,1392692,2019-03-01 03:00:00
79,79,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209673,1392711,2019-03-01 07:00:00
80,80,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209797,1392838,2019-03-01 11:00:00
81,81,A002,R051,02-00-01,59 ST,NQR456W,BMT,2019-03-01,REGULAR,6209986,1392887,2019-03-01 15:00:00


In [51]:
by_turnstile_date = mta.groupby(['c_a', 'unit', 'scp', 'station', mta.datetime.dt.date])

In [52]:
mta_daily = by_turnstile_date['entries'].min().reset_index()

In [53]:
mta_daily.describe([.95,.997])

,entries
count,4.494290e+05
mean,4.090301e+07
std,2.101619e+08
min,0.000000e+00
50%,2.183183e+06
95%,1.174405e+08
99.7%,1.924460e+09
max,2.129446e+09


In [54]:
mta_daily['daily_entries'] = mta_daily.groupby(['c_a', 'unit', 'scp', 'station'])['entries'].diff().shift(-1)

In [55]:
#dropping rows with negative daily entries
mta_daily = mta_daily[mta_daily.daily_entries >= 0]

In [56]:
#dropping rows with the top .3% of daily entries
mta_daily = mta_daily[mta_daily.daily_entries < mta_daily.daily_entries.quantile(q =.997)]

In [57]:
mta_daily

,c_a,unit,scp,station,datetime,entries,daily_entries
0,A002,R051,02-00-00,59 ST,2019-03-01,6962094,1528.0
1,A002,R051,02-00-00,59 ST,2019-03-02,6963622,741.0
2,A002,R051,02-00-00,59 ST,2019-03-03,6964363,541.0
3,A002,R051,02-00-00,59 ST,2019-03-04,6964904,1239.0
4,A002,R051,02-00-00,59 ST,2019-03-05,6966143,1481.0
...,...,...,...,...,...,...,...
449423,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-05-27,5554,0.0
449424,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-05-28,5554,0.0
449425,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-05-29,5554,0.0
449426,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-05-30,5554,0.0
